# Chargement des données

In [0]:
import numpy as np
import pandas as pd

X_train = np.load('/kaggle/input/classer-le-text/data_train.npy')
X_test = np.load('/kaggle/input/classer-le-text/data_test.npy')
df = pd.read_csv('/kaggle/input/classer-le-text/label_train.csv')
y_train = df['label'].to_numpy()

# Régression logistique de base 

In [ ]:
class LogisticRegression:
    def __init__(self, learning_rate=0.01, iterations=1000):
        # Initialisation avec le taux d'apprentissage et le nombre d'itérations
        self.learning_rate = learning_rate  
        self.iterations = iterations        
        self.W = None   
        self.b = None    

    def sigmoid(self, z):
        # Fonction d'activation sigmoid qui transforme les valeurs en probabilités entre 0 et 1
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        # Entraînement du modèle sur les données X avec les labels y
        self.n, self.d = X.shape          # n = nombre d'échantillons, d = nombre de features
        self.W = np.zeros(self.d)         # Initialisation des poids à zéro
        self.b = 0                        # Initialisation du biais à zéro

        for _ in range(self.iterations):
            self.update_weights(X, y)      # Mise à jour des poids à chaque itération
        
        return self

    def update_weights(self, X, y):
        # Calcul des prédictions actuelles avec la fonction sigmoid
        z = self.sigmoid(X.dot(self.W) + self.b)
        
        # Calcul des gradients pour les poids et le biais
        # dW = dérivée partielle par rapport aux poids
        dW = (1 / self.n) * np.dot(X.T, (z - y))
        # db = dérivée partielle par rapport au biais
        db = (1 / self.n) * np.sum(z - y)

        # Mise à jour des poids et du biais vec la descente de gradient
        self.W -= self.learning_rate * dW
        self.b -= self.learning_rate * db

    def predict(self, X):
        # Prédiction pour de nouvelles données
        z = self.sigmoid(X.dot(self.W) + self.b)
        return [1 if i > 0.5 else 0 for i in z]
    
model = LogisticRegression(learning_rate=0.01, iterations=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

predictions = pd.DataFrame({
    'ID': range(len(y_pred)),
    'label': y_pred
})

predictions.to_csv('predictions.csv', index=False)

# Pour tester localement

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

model = LogisticRegression(learning_rate=0.01, iterations=1000)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_val)

macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f"Validation Macro F1 Score: {macro_f1}")

# Régression logistique avec normalisation


In [ ]:
class LogisticRegression:
   def __init__(self, learning_rate=0.01, iterations=1000):
       # Initialisation des paramètres 
       self.learning_rate = learning_rate  
       self.iterations = iterations     
       self.W = None      
       self.b = None                      
       self.mean = None    
       self.std = None   

   def sigmoid(self, z):
       return 1 / (1 + np.exp(-z))

   def normalize_features(self, X):
       # Normalisation des features (standardisation)
       if self.mean is None or self.std is None:
           self.mean = np.mean(X, axis=0)  # Moyenne par colonne
           self.std = np.std(X, axis=0)    # Écart-type par colonne
       # Application de la normalisation
       # 1e-8 est ajouté pour éviter la division par zéro
       return (X - self.mean) / (self.std + 1e-8)
   
   def fit(self, X, y):
       # Entraînement du modèle
       X_normalized = self.normalize_features(X)  # Normalisation des données
       self.n, self.d = X_normalized.shape       # n = nb échantillons, d = nb features
       self.W = np.zeros(self.d)                 # Initialisation des poids à zéro
       self.b = 0                                # Initialisation du biais à zéro

       for _ in range(self.iterations):
           self.update_weights(X_normalized, y)

   def update_weights(self, X, y):
       # Calcul des prédictions actuelles
       z = self.sigmoid(X.dot(self.W) + self.b)
       
       # Calcul des gradients
       dW = (1 / self.n) * np.dot(X.T, (z - y))  # Gradient pour les poids
       db = (1 / self.n) * np.sum(z - y)         # Gradient pour le biais

       # Mise à jour des paramètres par descente de gradient
       self.W -= self.learning_rate * dW
       self.b -= self.learning_rate * db

   def predict(self, X):
       # Prédiction pour de nouvelles données
       X_normalized = self.normalize_features(X)  # Normalisation des nouvelles données
       z = self.sigmoid(X_normalized.dot(self.W) + self.b) 
       return [1 if i > 0.5 else 0 for i in z]
   
model = LogisticRegression(learning_rate=0.01, iterations=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

predictions = pd.DataFrame({
    'ID': range(len(y_pred)),
    'label': y_pred
})

predictions.to_csv('predictions1.csv', index=False)

# On peut utiliser le même code précédent pour tester localement

# Régression logistique avec normalisation des données et régularisation L2  

In [1]:
class LogisticRegression:
   def __init__(self, learning_rate=0.01, iterations=1000, lambda_reg=0.01):
       self.learning_rate = learning_rate  
       self.iterations = iterations       
       self.lambda_reg = lambda_reg       
       self.W = None                      
       self.b = None               
       self.mean = None        
       self.std = None               

   def sigmoid(self, z):
       return 1 / (1 + np.exp(-z))

   def normalize_features(self, X):
       # Normalisation des features
       if self.mean is None or self.std is None:
           self.mean = np.mean(X, axis=0)
           self.std = np.std(X, axis=0)
       return (X - self.mean) / (self.std + 1e-8)  
   
   def fit(self, X, y):
       # Entraînement du modèle
       X_normalized = self.normalize_features(X)
       self.n, self.d = X_normalized.shape
       self.W = np.zeros(self.d)
       self.b = 0
       
       self.losses = []  # Stockage des pertes pour suivre la convergence
       
       for _ in range(self.iterations):
           self.update_weights(X_normalized, y)
       
       return self
   
   def update_weights(self, X, y):
       # Calcul des prédictions
       z = self.sigmoid(X.dot(self.W) + self.b)

       # Calcul de la fonction de perte (cross-entropy avec régularisation L2)
       # 1e-15 ajouté pour éviter log(0)
       loss = -np.mean(y * np.log(z + 1e-15) + 
                     (1 - y) * np.log(1 - z + 1e-15))
       # Terme de régularisation L2
       reg_loss = self.lambda_reg * np.sum(self.W ** 2) / (2 * self.n)
       # Perte totale
       total_loss = loss + reg_loss
       self.losses.append(total_loss)  # Sauvegarde de la perte

       # Calcul des gradients avec régularisation
       dW = (1 / self.n) * np.dot(X.T, (z - y)) + \
            (self.lambda_reg * self.W / self.n)  # Gradient régularisé
       db = (1 / self.n) * np.sum(z - y)

       # Mise à jour des paramètres
       self.W -= self.learning_rate * dW
       self.b -= self.learning_rate * db
       
   def predict(self, X):
       X_normalized = self.normalize_features(X)
       z = self.sigmoid(X_normalized.dot(self.W) + self.b)
       return [1 if i > 0.5 else 0 for i in z]
   
model = LogisticRegression(learning_rate=0.01, iterations=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

predictions = pd.DataFrame({
    'ID': range(len(y_pred)),
    'label': y_pred
})

predictions.to_csv('predictions2.csv', index=False)